## End-2-End Pipeline from scratch

?torch.tensor(label)
?transforms.RandomResizedCrop(224),


In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" --header="Cookie: _ga=GA1.3.723045802.1563092728" --header="Connection: keep-alive" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/14774/536888/all.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1564818803&Signature=cGZ%2B9gUtXgqr0IAuP1dasVxTN1hm9EGqAz8XTLpJFBw92J%2FMacrdfarqVPtoWNPVc47zCMFXh8JO7OqpF03E%2BR7tWOqFk6SEH2mLvdgjKAaoq%2FPx4DLpqdC6CTh8YKqKKGJzZFcTZ5aFmocUgTTBhyaHfjOHoqR%2Fgo4JE663ixE1gSwciXCt%2BJjRNXqRL3drzKRRCDD1oM3xDuVBjHob5HboPoT2MCi08ZnkwkHId05abOSz%2FxnAMoyrsJSwGM5jsNF7LjzCETcqRQBkCwAWFJX46hnWzp3WBnN0uzOirqDt9FU7qH6HAkVa4Yhty%2Bd%2BHegB8Ae3KihQGMmotrh%2FlA%3D%3D&response-content-disposition=attachment%3B+filename%3Daptos2019-blindness-detection.zip" -O "aptos2019-blindness-detection.zip" -c
!unzip -q aptos2019-blindness-detection.zip

In [0]:
!rm aptos2019-blindness-detection.zip

In [0]:
%%bash
mkdir test_images
mkdir train_images
unzip -q test_images.zip -d test_images
unzip -q train_images.zip -d train_images

In [0]:
!rm train_images.zip
!rm test_images.zip

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" --header="Cookie: _ga=GA1.3.723045802.1563092728" --header="Connection: keep-alive" "https://storage.googleapis.com/kaggle-datasets/27833/35481/pytorch-pretrained-models.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1564909775&Signature=RLj%2BFTlMVXG81iFn3Rt2cT6DeuGLxK%2BSVmQNPWMQEch66Yyca85uFfs4eF2fj5PuU6ACjtgIBcwRv2S2tMAlsibjQy4A7uWcMFf6r4kcm5GICgwIHvTUTUQrA9GGvpkXVAPIDIN4MTZk%2FQ0f1UBfJ6Eu%2FitKwfEh2%2F5HA0cftq0AKR%2F39IP5OOHVp7Bl7pdThjzpan%2B%2FviQltz6WuwuD0unWhu1JzT4GzeBC5XZBHWngrh7uPBcOMsf%2BcHG5ZNZll2HmXMzx4wECosTl7Yd3NUU16NFDZwFRTTmr2gE3J4vh5nUKc4Slkso0mmSKd1OvC5vyJK6DIIxHmZizFONiOA%3D%3D" -O "pytorch-pretrained-models.zip" -c

In [0]:
import os
os.listdir()

In [0]:
!unzip -q pytorch-pretrained-models.zip

In [0]:
labels_frame.head()

In [0]:
#Data Loading and Preprocessing:
'''
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
'''
import pandas as pd

aptos_frame = pd.read_csv('train.csv')
n = 65
img_name = aptos_frame.iloc[n, 0] + '.png'
label = aptos_frame.iloc[n,1]

print('Image name: {}'.format(img_name))
print('Label: {}'.format(label))

In [0]:
import matplotlib.pyplot as plt
import os
from skimage import io, transform

def show_aptos(image, label):
  """Show image with label"""
  plt.imshow(image)
  #ax = plt.subplot()
  plt.title('Label {}'.format(label))
  #print('Label {}'.format(label))
  plt.pause(0.001) #pause a bit so that plots are updated

plt.figure()
show_aptos(io.imread(os.path.join('train_images/', img_name)), label)

plt.show()

In [0]:
#Dataset Class:

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

class APTOSDataset(Dataset):
  """APTOS 2019 Blindness Detection Dataset."""

  def __init__(self, csv_file, root_dir, transform=None):
    """
    Args:
        csv_file (string): Path to the csv file with labels.
        root_dir (string): Directory with all the images.
        transform (callable, optional): Optional transform to be applied on a sample.
    """
    self.label_frame = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.label_frame)

  def __getitem__(self, idx):
    img_name = os.path.join(self.root_dir, self.label_frame.iloc[idx, 0]) + '.png'
    image = io.imread(img_name)
    label = self.label_frame.iloc[idx, 1]
    sample = {'image': image, 'label': label}

    if self.transform:
      sample = self.transform(sample)

    return sample

In [0]:
aptos_dataset = APTOSDataset(csv_file='train.csv', root_dir='train_images/')

fig = plt.figure()

for i in range(len(aptos_dataset)):
    sample = aptos_dataset[i]

    print(i, sample['image'].shape, sample['label'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_aptos(**sample)

    if i == 3:
        plt.show()
        break

In [0]:
#Transforms

import numpy as np

class Rescale(object):
  """Rescale the image in a sample to a given size.

  Args:
      output_size (tuple or int): Desired output size. If tuple, output is
      matched to output_size. If int, smaller of image edges is matched to
      output_size keeping aspect ratio the same.
  """
  
  def __init__(self,  output_size):
    assert isinstance(output_size, (int, tuple))
    self.output_size = output_size

  def __call__(self, sample):
    image, label = sample['image'], sample['label']

    h, w = image.shape[:2]
    if isinstance(self.output_size, int):
      if h > w:
        new_h, new_w = self.output_size * h / w, self.output_size
      else:
        new_h, new_w = self.output_size, self.output_size * w / h

    else:
        new_h, new_w = self.output_size
        
    new_h, new_w = int(new_h), int(new_w)
    
    img = transform.resize(image, (new_h, new_w))
    
    # labels will have no effect
    
    return {'image' : img, 'label' : label}


class RandomCrop(object):
  """Crop randomly the image in a sample

  Args:
      output_size (tuple or int): Desired output size. If int, square crop
      is made.
  """

  def __init__(self, output_size):
    assert isinstance(output_size, (int, tuple))
    if isinstance(output_size, int):
      self.output_size = (output_size, output_size)

    else:
      assert len(output_size) == 2
      self.output_size = output_size

  def __call__(self, sample):
    image, label = sample['image'], sample['label']
    
    h, w = image.shape[:2]
    new_h, new_w = self.output_size

    top = np.random.randint(0, h - new_h)
    left = np.random.randint(0, w - new_w)
    
    image = image[top: top + new_h,
                  left: left + new_w]

    return {'image': image, 'label': label}

class ToTensor(object):
  """Convert ndarrays in sample to Tensors."""
  def __call__(self, sample):
    image, label = sample['image'], sample['label']

    # swap color axis because
    # numpy image: H x W x C
    # torch image: C X H X W
    image = image.transpose((2, 0, 1))

    return {'image': torch.from_numpy(image),
            'label': torch.tensor(label)}

In [0]:
# Compose Transforms
scale = Rescale(256)
crop = RandomCrop(224)
composed = transforms.Compose([Rescale(256), RandomCrop(224)])

# Apply each of the above transforms on sample.
fig = plt.figure()
sample = aptos_dataset[65]
for i, tsfrm in enumerate([scale, crop, composed]):
  transformed_sample = tsfrm(sample)

  ax = plt.subplot(1, 3, i + 1)
  plt.tight_layout()
  ax.set_title(type(tsfrm).__name__)
  show_aptos(**transformed_sample)

In [0]:
# iterating through the dataset

transformed_dataset = APTOSDataset(csv_file = 'train.csv',
                                   root_dir = 'train_images/',
                                   transform = transforms.Compose([
                                                                   Rescale(256),
                                                                   RandomCrop(224),
                                                                   ToTensor()
                                                                   ]))

for i in range(len(transformed_dataset)):
  sample = transformed_dataset[i]

  print(i, sample['image'].size(), sample['label'].size())

  if i == 3:
    break

In [0]:
"""
Batching the data
Shuffling the data
Load the data in parallel using multiprocessing workers
"""
dataloader = DataLoader(transformed_dataset, batch_size = 4,
                        shuffle=True, num_workers=4)

# Helper function to show a batch
def show_aptos_batch(sample_batched):
  """Show image with label for a batch of samples."""
  images_batch, labels_batch = \
          sample_batched['image'], sample_batched['label']
  batch_size = len(images_batch)
  im_size = images_batch.size(2)
  grid_border_size = 2

  grid = utils.make_grid(images_batch)
  plt.imshow(grid.numpy().transpose((1, 2, 0)))

  for i in range(batch_size):
    plt.title('Batch from dataloader')
  

for i_batch, sample_batched in enumerate(dataloader):
  print(i_batch, sample_batched['image'].size(),
        sample_batched['label'].size())
  
  # observe 4th batch and stop.
  if i_batch == 3:
    plt.figure()
    show_aptos_batch(sample_batched)
    plt.axis('off')
    plt.ioff()
    plt.show()
    break

In [0]:
# transfer learning 
'''
https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
'''
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode



In [0]:
# data augmentation and normalization for training
# just normalization for validation

data_transforms = {
    'train' : transforms.Compose([
        Rescale(256),
        RandomCrop(224),
        ToTensor()
        #transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        #transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val' : transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
from torch.utils.data import sampler

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples
  
NUM_TRAIN = 2930
NUM_VAL = 732

dataset_sizes = {'train' : NUM_TRAIN, 'val' : NUM_VAL}

train_dataset = APTOSDataset(csv_file = 'train.csv',
                                   root_dir = 'train_images/',
                                   transform = data_transforms['train'])

val_dataset = APTOSDataset(csv_file = 'train.csv',
                                   root_dir = 'train_images/',
                                   transform = data_transforms['train'])

image_datasets = {'train' : train_dataset, 'val' : val_dataset}

loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=4, num_workers=4, sampler=ChunkSampler(NUM_TRAIN, 0))

loader_val = torch.utils.data.DataLoader(val_dataset, batch_size=4, num_workers=4, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

dataloaders = {'train' : loader_train, 'val' : loader_val}

In [0]:
def imshow(inp, title=None):
  """Imshow for Tensor."""
  inp = inp.numpy().transpose((1, 2, 0))
  mean = np.array([0.485, 0.456, 0.406])
  std = np.array([0.299, 0.224, 0.225])
  #inp = std*inp+mean
  #inp = inp.clip(inp, 0, 1)
  plt.imshow(inp)
  if title is not None:
    plt.title(title)
  plt.pause(0.001) # pause a bit so that plots are updated


# Get a batch of training data
sample_batched = next(iter(dataloaders['train']))
inputs = sample_batched['image']
classes = sample_batched['label']

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[str(x) for x in classes])

In [0]:
'''
Training the model
 # Scheduling the learning rate
 # Saving the best model
'''
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
      if phase == 'train':
        scheduler.step()
        model.train() # set the model to training mode
      else:
        model.eval()

      running_loss = 0.0
      running_corrects = 0

      # Iterate over data.
      for sample_batched in dataloaders[phase]:
        inputs = sample_batched['image'].to(device, dtype=torch.float)
        labels = sample_batched['label'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history if only in train
        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          loss = criterion(outputs, labels).cpu().numpy()

          # backward + optimize only if in trianing phase
          if phase == 'train':
            loss.backward()
            optimizer.step()
            
        # statistics
        running_loss += loss.items() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects.double() / dataset_sizes[phase]

      print('{}  Loss: {:.4f} Acc: {:.4f}'.format(
          phase, epoch_loss, epoch_acc))
      
      # deep copy the model
      if phase == 'val' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
    
    print()

  time_elapsed = time.time() - since
  print('Training complete in {:0f}m {:.0f}s'.format(
      time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  # load the best model weights
  model.load_state_dict(best_model_wts)
  return model

In [0]:
# visualize the model predicitons
def visualize_model(model, num_images=6):
  was_training = model.training
  model.eval()
  images_so_far = 0
  fig = plt.figure()

  with torch.no_grad():
    for i, sample_batched in enumerate(dataloaders['val']):
      inputs = sample_batched['image'].to(device)
      labels = sample_batched['label'].to(device)

      outputs = model(inputs)
      _, preds = torch.max(outputs, 1)

      for j in range(inputs.size()[0]):
        images_so_far += 1
        ax = plt.subplot(num_images//2, 2, images_so_far)
        ax.axis('off')
        ax.set_title('predicted: {}'.format(str(preds[j])))
        imshow(inputs.cpu().data[j])

        if images_so_far == num_images:
          model.train(mode=was_training)
          return
    model.train(mode=was_training)  

In [0]:
model_ft = models.resnet101(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(2048, 5)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [0]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=4)

In [0]:
#importing libraries needed

import pandas as pd

import time
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm

from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import os

device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [0]:
#defining dataset class:
'''
1. Training Dataset
2. Testing Dataset
'''

from torch.utils.data import Dataset
import pandas as pd
import os
import torch
from PIL import Image, ImageFile
from torchvision import transforms


class RetinopathyDatasetTrain(Dataset):

    def __init__(self, csv_file):

        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('train_images', self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = image.resize((256, 256), resample=Image.BILINEAR)
        label = torch.tensor(self.data.loc[idx, 'diagnosis'])
        return {'image': transforms.ToTensor()(image),
                'labels': label
                }


class RetinopathyDatasetTest(Dataset):
    def __init__(self, csv_file, transform):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('test_images', self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = self.transform(image)
        return {'image': image}

In [0]:
from torch.utils.data import sampler

class ChunkSampler(sampler.Sampler):
    """Samples elements sequentially from some offset. 
    Arguments:
        num_samples: # of desired datapoints
        start: offset where we should start selecting from
    """
    def __init__(self, num_samples, start = 0):
        self.num_samples = num_samples
        self.start = start

    def __iter__(self):
        return iter(range(self.start, self.start + self.num_samples))

    def __len__(self):
        return self.num_samples
      
      
NUM_TRAIN = 2930
NUM_VAL = 732

train_dataset = RetinopathyDatasetTrain(csv_file='train.csv')

loader_train = torch.utils.data.DataLoader(train_dataset, batch_size=16, num_workers=4, sampler=ChunkSampler(NUM_TRAIN, 0))

#val_dataset = RetinopathyDatasetTrain(csv_file='train.csv')

loader_val = torch.utils.data.DataLoader(train_dataset, batch_size=16, num_workers=4, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

'''
cifar10_train = dset.CIFAR10(PATH_TO_CIFAR, train=True, download=False,
                           transform=T.ToTensor())
loader_train = DataLoader(cifar10_train, batch_size=64, sampler=ChunkSampler(NUM_TRAIN, 0))

cifar10_val = dset.CIFAR10(PATH_TO_CIFAR, train=True, download=False,
                           transform=T.ToTensor())
loader_val = DataLoader(cifar10_val, batch_size=64, sampler=ChunkSampler(NUM_VAL, NUM_TRAIN))

cifar10_test = dset.CIFAR10(PATH_TO_CIFAR, train=False, download=False,
                          transform=T.ToTensor())
loader_test = DataLoader(cifar10_test, batch_size=64)
'''

In [0]:
from os import *
import matplotlib.pyplot as plt
from numpy import *
from PIL import Image
from pandas import *

In [0]:
len(train_dataset)

In [0]:
fig = plt.figure()

for i in range(NUM_TRAIN):
  sample = train_dataset[i]
  
  print(i, sample['image'].shape, sample['labels'].shape)
  
  if i == 3:
    break

In [0]:
fig = plt.figure()

for i in range(NUM_TRAIN):
  sample = train_dataset[i]
  print(i, sample['image'].shape, sample['labels'].shape)
  image = sample['image']
  fig = plt.figure()
  plt.imshow(np.transpose(image, (1, 2, 0))) 

  
  if i == 3:
    plt.show()
    break

In [0]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    #img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(loader_train)
sample = dataiter.next()
images = sample['image']
labels = sample['labels']

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % labels[j].numpy() for j in range(4)))

In [0]:
#take a look at data:
fig = plt.figure(figsize=(25, 4))
dataiter = iter(loader_train)
for i in range(20):
  sample = dataiter.next()
  images = sample['image']
  labels = sample['labels']
  ax = fig.add_subplot(2, 20//2, labels+1, xticks=[], yticks=[])
  plt.imshow(images)
  ax.set_title(f'Label: {labels}')

In [0]:
for t, sample in enumerate(loader_train):
  print(t, sample['image'].shape, sample['labels'].shape)
  if t>5: break

In [0]:
face_dataset = RetinopathyDatasetTrain(csv_file='train.csv')


for i in range(len(face_dataset)):
    sample = face_dataset[i]

    print(i, sample['image'].shape, sample['labels'].shape)
    
    if i == 3:
      break

In [0]:
model = torchvision.models.resnet101(pretrained=False)
model.load_state_dict(torch.load("resnet101-5d3b4d8f.pth"))
num_features = model.fc.in_features
model.fc = nn.Linear(2048, 5)

model = model.to(device)

In [0]:
plist = [
         {'params': model.layer4.parameters(), 'lr': 1e-4, 'weight': 0.001},
         {'params': model.fc.parameters(), 'lr': 1e-3}
         ]

optimizer = optim.Adam(plist, lr=0.001)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10)
criterion = nn.CrossEntropyLoss()

In [0]:
since = time.time()
criterion = nn.CrossEntropyLoss()
num_epochs = 3
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    scheduler.step()
    model.train()
    running_loss = 0.0
    tk0 = tqdm(loader_train, total=int(len(loader_train)))
    counter = 0
    for bi, d in enumerate(tk0):
        inputs = d["image"]
        labels = d["labels"]
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        counter += 1
        tk0.set_postfix(loss=(running_loss / (counter * loader_train.batch_size)))
    epoch_loss = running_loss / len(loader_train)
    print('Training Loss: {:.4f}'.format(epoch_loss))

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
torch.save(model.state_dict(), "model.bin")

In [0]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_dataset = RetinopathyDatasetTest(csv_file='sample_submission.csv',
                                      transform=test_transform)

In [0]:
import numpy as np
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

model.eval()
diagnosis = []
with torch.no_grad():
  tk0 = tqdm(test_data_loader, total=int(len(test_data_loader)))
  for bi, d in enumerate(tk0):
    inputs = d["image"].to(device)
    output = model(inputs)
    pred = np.argmax(output.detach().cpu().numpy(), axis=1)
    for preds in pred:
      diagnosis.append(preds)
    #_, preds = output.data.cpu().max(1)
    #preds = output.detach().cpu().squeeze().numpy().ravel().reshape(-1, 1)
    #print(pred)
    #pred = output.data.max(1, keepdim=True)[1].cpu().numpy().ravel().reshape(-1, 1)
    #print(pred)
    #diagnosis.append(pred)

In [0]:
len(diagnosis)

In [0]:
sample = pd.read_csv('sample_submission.csv')
sample.head()

In [0]:
sample.diagnosis = diagnosis
sample.head()